In [ ]:
%autoreload 2|

In [ ]:
from collections import defaultdict
from argparse import Namespace
import copy
import difflib
import gzip
import itertools
import os
import pickle
import sys
import typing

from IPython.display import display, Markdown, HTML
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tatsu
import tatsu.ast
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import tqdm
import sklearn
from sklearn.model_selection import GridSearchCV, train_test_split, KFold
from sklearn.pipeline import Pipeline

sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../src'))
from src import fitness_energy_utils as utils
from src.fitness_energy_utils import NON_FEATURE_COLUMNS
from src.fitness_features import build_fitness_featurizer, ASTFitnessFeaturizer
from src.ast_counter_sampler import *
from src.ast_mcmc_regrowth import *
from src import ast_printer

In [ ]:
DEFAULT_ARGS = argparse.Namespace(
    grammar_file=os.path.join('..', DEFAULT_GRAMMAR_FILE),
    parse_counter=False,
    counter_output_path=os.path.join('..', DEFAULT_COUNTER_OUTPUT_PATH),
    random_seed=DEFUALT_RANDOM_SEED,
)

In [ ]:
import ast_crossover_sampler
from ast_counter_sampler import DEFAULT_GRAMMAR_FILE
from ast_utils import cached_load_and_parse_games_from_file
grammar = open(DEFAULT_ARGS.grammar_file).read()
grammar_parser = tatsu.compile(grammar)
counter = parse_or_load_counter(DEFAULT_ARGS, grammar_parser)
sampler = ASTSampler(grammar_parser, counter, seed=DEFAULT_ARGS.random_seed)
asts = [ast for ast in cached_load_and_parse_games_from_file('../dsl/interactive-beta.pddl', 
    grammar_parser, False, relative_path='..')]


In [ ]:
crossover_sampler = ast_crossover_sampler.CrossoverSampler(
    ast_crossover_sampler.CrossoverType.SAME_RULE, 
    asts[1:],
    sampler,
    DEFAULT_ARGS.random_seed,
    use_tqdm=True,
)
crossover_sampler.set_source_ast(asts[0])

In [1]:
samples = []
for i in range(10):
    s = crossover_sampler.sample(i, crossover_key_to_use='variable_list')
    samples.append(s)
    print(ast_printer.ast_to_string(s, '\n'))
    print('=' * 80)

NameError: name 'crossover_sampler' is not defined

In [ ]:
html_diff = difflib.HtmlDiff(wrapcolumn=60)

def diff_by_index(index: int):
    src = crossover_sampler.source_ast
    mod = samples[index]
    src_str = ast_printer.ast_to_string(src, '\n')
    mod_str = ast_printer.ast_to_string(mod, '\n')

    diff = html_diff.make_file(src_str.splitlines(), mod_str.splitlines())  #, context=True, numlines=0)
    display(HTML(diff))

In [ ]:
diff_by_index(3)